## Test the environment `HerdEnv` of `herd_env.py`

In [1]:
import numpy as np
import tensorflow as tf
from tf_agents.environments import utils
from tf_agents.trajectories.time_step import StepType
from tf_agents.trajectories import TimeStep
from tf_agents.policies import scripted_py_policy
from tf_agents.policies import random_py_policy
from tf_agents.metrics import py_metrics
from tf_agents.drivers import py_driver
from tf_agents.utils import common

In [2]:
from herd_env import HerdEnv

In [3]:
# sanity check
henv_val = HerdEnv(herd_sizes = [64,64], rand_recovery_prob = 0.1, rand_infection_prob = 0.05)
utils.validate_py_environment(henv_val, episodes=10)

In [4]:
# create Herd Environment instance to be trained for
max_episode_length=100
num_herds=2
henv = HerdEnv(herd_sizes = [32,32], expected_episode_length=50., max_episode_length=max_episode_length,
               rand_recovery_prob = 0.04, rand_infection_prob = 0.05)

In [5]:
# show interor values of environment
time_step = henv.reset()
print(time_step)
cumulative_reward = time_step.reward
finished = False

while not finished:
  time_step = henv.step(0) # do nothing
  s = henv.get_state()
  print("state: ", s, "observation: ", time_step.observation, "\treward: ", time_step.reward)
  cumulative_reward += time_step.reward
  if time_step.step_type == StepType.LAST:
    finished = True

print('Final Reward = ', cumulative_reward)

TimeStep(
{'discount': 1.0,
 'observation': array([0., 0.], dtype=float32),
 'reward': 0.0,
 'step_type': array(0, dtype=int32)})
state:  [2 1 1 1] observation:  [0.01 0.01] 	reward:  0.0
state:  [1 1 2 2] observation:  [0.02 0.02] 	reward:  0.0
state:  [2 2 3 3] observation:  [0.03 0.03] 	reward:  0.0
state:  [2 3 4 4] observation:  [0.04 0.04] 	reward:  0.0
state:  [3 3 5 5] observation:  [0.05 0.05] 	reward:  0.0
state:  [4 6 6 6] observation:  [0.06 0.06] 	reward:  0.0
state:  [6 8 7 7] observation:  [0.07 0.07] 	reward:  0.0
state:  [9 9 8 8] observation:  [0.08 0.08] 	reward:  0.0
state:  [9 9 9 9] observation:  [0.09 0.09] 	reward:  0.0
state:  [12 10 10 10] observation:  [0.1 0.1] 	reward:  0.0
state:  [12 11 11 11] observation:  [0.11 0.11] 	reward:  0.0
state:  [13 10 12 12] observation:  [0.12 0.12] 	reward:  0.0
state:  [15 12 13 13] observation:  [0.13 0.13] 	reward:  -540.0
Final Reward =  -540.0


In [6]:
action_spec = henv.action_spec()
ts_spec = henv.time_step_spec()
print("action spec:\n", action_spec, "\n\ntime step spec:\n", ts_spec)

action spec:
 BoundedArraySpec(shape=(), dtype=dtype('int32'), name='action', minimum=0, maximum=3) 

time step spec:
 TimeStep(
{'discount': BoundedArraySpec(shape=(), dtype=dtype('float32'), name='discount', minimum=0.0, maximum=1.0),
 'observation': BoundedArraySpec(shape=(2,), dtype=dtype('float32'), name='observation', minimum=0.0, maximum=1.0),
 'reward': ArraySpec(shape=(), dtype=dtype('float32'), name='reward'),
 'step_type': ArraySpec(shape=(), dtype=dtype('int32'), name='step_type')})


### Define scripted policies

In [7]:
# do nothing policy: cull never
action_script0 = [(max_episode_length, 0)]

# cull first herd every 10th step and second herd every 20th step
action_script1 = [(9, 0), 
                 (1, 1),
                 (9, 0), 
                 (1, 3)] * int(1+max_episode_length/20)

manual_scripted_policy0 = scripted_py_policy.ScriptedPyPolicy(
    time_step_spec=ts_spec,
    action_spec=action_spec,
    action_script=action_script0)

manual_scripted_policy1 = scripted_py_policy.ScriptedPyPolicy(
    time_step_spec=ts_spec,
    action_spec=action_spec,
    action_script=action_script1)

init_policy_state = manual_scripted_policy0.get_initial_state()

In [8]:
policy_state =  init_policy_state
ts0 = henv.reset()
for _ in range(21):
    action_step = manual_scripted_policy1.action(ts0, policy_state)
    policy_state = action_step.state
    print("action=", action_step.action, "\tpolicy_state", policy_state)
policy_state = manual_scripted_policy1.get_initial_state()

action= 0 	policy_state [0, 1]
action= 0 	policy_state [0, 2]
action= 0 	policy_state [0, 3]
action= 0 	policy_state [0, 4]
action= 0 	policy_state [0, 5]
action= 0 	policy_state [0, 6]
action= 0 	policy_state [0, 7]
action= 0 	policy_state [0, 8]
action= 0 	policy_state [0, 9]
action= 1 	policy_state [1, 1]
action= 0 	policy_state [2, 1]
action= 0 	policy_state [2, 2]
action= 0 	policy_state [2, 3]
action= 0 	policy_state [2, 4]
action= 0 	policy_state [2, 5]
action= 0 	policy_state [2, 6]
action= 0 	policy_state [2, 7]
action= 0 	policy_state [2, 8]
action= 0 	policy_state [2, 9]
action= 3 	policy_state [3, 1]
action= 0 	policy_state [4, 1]


### ... and a random policy

In [9]:
random_policy = random_py_policy.RandomPyPolicy(time_step_spec=ts_spec, action_spec=action_spec)

## Drive a rollout

In [10]:
def compute_avg_return(environment, policy, num_episodes=50, verbose=False):
  total_return = 0.0
  cullsteps = 0
  for e in range(num_episodes):

    time_step = environment.reset()
    if isinstance(policy, scripted_py_policy.ScriptedPyPolicy):
        policy_state = policy.get_initial_state() # remember where in the script we were
    else:
        policy_state = None # other policies without memory
    episode_return = 0.0
    i=0
    while not time_step.is_last():
        i+=1
        action_step = policy.action(time_step, policy_state)
        if action_step.action > 0:
            cullsteps += 1
        policy_state = action_step.state
        time_step = environment.step(action_step.action)
        if isinstance(environment, HerdEnv):
            state = environment.get_state()
        else:
            state = None # TF environment from wrapper does not have get_state()
        episode_return += time_step.reward
        if verbose:
            print (f"episode {e:>2} step{i:>3} action: ", action_step.action, "state=", state, "obs=", time_step.observation, "reward=", time_step.reward)
    total_return += episode_return

  avg_return = total_return / num_episodes
  cullsteps /= num_episodes
  return avg_return, cullsteps

In [11]:
random_reward, cullsteps = compute_avg_return(henv, random_policy)
print (f"average return of random policy: {random_reward:.3f} avg steps with culls per episode: {cullsteps}")

average return of random policy: -3060.000 avg steps with culls per episode: 36.2


In [12]:
# show states for one rollout of second scripted policy
compute_avg_return(henv, manual_scripted_policy1, num_episodes=1, verbose=True)

episode  0 step  1 action:  0 state= [2 1 1 1] obs= [0.01 0.01] reward= 0.0
episode  0 step  2 action:  0 state= [3 5 2 2] obs= [0.02 0.02] reward= 0.0
episode  0 step  3 action:  0 state= [3 6 3 3] obs= [0.03 0.03] reward= 0.0
episode  0 step  4 action:  0 state= [5 8 4 4] obs= [0.04 0.04] reward= 0.0
episode  0 step  5 action:  0 state= [6 8 5 5] obs= [0.05 0.05] reward= 0.0
episode  0 step  6 action:  0 state= [8 9 6 6] obs= [0.06 0.06] reward= 0.0
episode  0 step  7 action:  0 state= [ 8 12  7  7] obs= [0.07 0.07] reward= 0.0
episode  0 step  8 action:  0 state= [ 9 14  8  8] obs= [0.08 0.08] reward= 0.0
episode  0 step  9 action:  0 state= [10 16  9  9] obs= [0.09 0.09] reward= 0.0
episode  0 step 10 action:  1 state= [ 0 16  0 10] obs= [0.  0.1] reward= -62.0
episode  0 step 11 action:  0 state= [ 3 17  1 11] obs= [0.01 0.11] reward= 0.0
episode  0 step 12 action:  0 state= [ 5 15  2 12] obs= [0.02 0.12] reward= 0.0
episode  0 step 13 action:  0 state= [ 5 14  3 13] obs= [0.03 0.

(-582.0, 1.0)

In [13]:
manual_reward0, cullsteps = compute_avg_return(henv, manual_scripted_policy0, num_episodes=100)
print (f"average return of do-nothing-policy: {manual_reward0:.3f} avg culls {cullsteps}")
manual_reward1, cullsteps = compute_avg_return(henv, manual_scripted_policy1, num_episodes=100)
print (f"average return of manual policy: {manual_reward1:.3f} avg culls {cullsteps}")

average return of do-nothing-policy: -687.600 avg culls 0.0
average return of manual policy: -689.460 avg culls 4.38


### Train a Deep-Q Agent

In [14]:
from tf_agents.networks.sequential import Sequential
from tensorflow.keras.layers import Dense
from tf_agents.agents.dqn import dqn_agent
from tf_agents.environments import tf_py_environment
from tf_agents.replay_buffers import tf_uniform_replay_buffer
from tf_agents.trajectories import trajectory

In [15]:
num_iterations = 30000
replay_buffer_max_length = 10000
batch_size = 16
num_eval_episodes = 20
initial_collect_steps = 100
collect_steps_per_iteration = 32
log_interval = 100
eval_interval = 400

In [16]:
# make actor network simple
num_actions = 2**num_herds # this does not scale, obviously
kernel_initializer=tf.keras.initializers.RandomUniform(minval=-0.03, maxval=0.03)
q_net = Sequential([Dense(4, activation=None,
                          kernel_initializer = kernel_initializer)])

In [17]:
train_step_counter = tf.Variable(0)

train_env = tf_py_environment.TFPyEnvironment(henv)
eval_env = tf_py_environment.TFPyEnvironment(henv)

agent = dqn_agent.DqnAgent(
    train_env.time_step_spec(),
    train_env.action_spec(),
    q_network=q_net,
    boltzmann_temperature = 0.005,
    epsilon_greedy = None,
    optimizer=tf.keras.optimizers.Adam(learning_rate = 1e-3),
    td_errors_loss_fn=common.element_wise_squared_loss,
    train_step_counter=train_step_counter)

agent.initialize()

In [18]:
agent.policy.trainable_variables

(<tf.Variable 'sequential/dense/kernel:0' shape=(2, 4) dtype=float32, numpy=
 array([[ 0.00947275,  0.00769239, -0.02492321, -0.02975761],
        [ 0.02453793, -0.02131323,  0.00301866,  0.01287598]],
       dtype=float32)>,
 <tf.Variable 'sequential/dense/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>)

In [19]:
# manually initialize a reasonably good policy: kill both herds if the sum of observations is > 1
W = np.array([[0, 0 ,0, 2],[0, 0, 0, 2,]])
b = np.array([1, 0, 0, 0])
q_net.layers[0].set_weights([W,b])
agent.policy.trainable_variables

(<tf.Variable 'sequential/dense/kernel:0' shape=(2, 4) dtype=float32, numpy=
 array([[0., 0., 0., 2.],
        [0., 0., 0., 2.]], dtype=float32)>,
 <tf.Variable 'sequential/dense/bias:0' shape=(4,) dtype=float32, numpy=array([1., 0., 0., 0.], dtype=float32)>)

In [20]:
replay_buffer = tf_uniform_replay_buffer.TFUniformReplayBuffer(
    data_spec=agent.collect_data_spec,
    batch_size=train_env.batch_size,
    max_length=replay_buffer_max_length)

In [21]:
agent.collect_data_spec._fields

('step_type',
 'observation',
 'action',
 'policy_info',
 'next_step_type',
 'reward',
 'discount')

In [22]:
def collect_step(environment, policy, buffer):
  time_step = environment.current_time_step()
  action_step = policy.action(time_step)
  next_time_step = environment.step(action_step.action)
  traj = trajectory.from_transition(time_step, action_step, next_time_step)

  # Add trajectory to the replay buffer
  buffer.add_batch(traj)

def collect_data(env, policy, buffer, steps):
  for _ in range(steps):
    collect_step(env, policy, buffer)

collect_data(train_env, random_policy, replay_buffer, initial_collect_steps)

In [23]:
# Dataset generates trajectories with shape [Bx2x...]
dataset = replay_buffer.as_dataset(
    num_parallel_calls=3, 
    sample_batch_size=batch_size, 
    num_steps=2).prefetch(3)

iterator = iter(dataset)
dataset

Instructions for updating:
Use `tf.data.Dataset.scan(...) instead


Instructions for updating:
Use `tf.data.Dataset.scan(...) instead


Instructions for updating:
Use `as_dataset(..., single_deterministic_pass=False) instead.


Instructions for updating:
Use `as_dataset(..., single_deterministic_pass=False) instead.


<PrefetchDataset shapes: (Trajectory(
{action: (16, 2),
 discount: (16, 2),
 next_step_type: (16, 2),
 observation: (16, 2, 2),
 policy_info: (),
 reward: (16, 2),
 step_type: (16, 2)}), BufferInfo(ids=(16, 2), probabilities=(16,))), types: (Trajectory(
{action: tf.int32,
 discount: tf.float32,
 next_step_type: tf.int32,
 observation: tf.float32,
 policy_info: (),
 reward: tf.float32,
 step_type: tf.int32}), BufferInfo(ids=tf.int64, probabilities=tf.float32))>

In [24]:

# Reset the train step
agent.train_step_counter.assign(0)

# Evaluate the agent's policy once before training.
avg_return = compute_avg_return(eval_env, agent.policy, num_eval_episodes)
returns = [avg_return]

for _ in range(num_iterations):

  # Collect a few steps using collect_policy and save to the replay buffer.
  collect_data(train_env, agent.collect_policy, replay_buffer, collect_steps_per_iteration)

  # Sample a batch of data from the buffer and update the agent's network.
  experience, unused_info = next(iterator)
  train_loss = agent.train(experience).loss

  step = agent.train_step_counter.numpy()

  if step <= 1 or step % log_interval == 0:
    print('step = {0:4>}: loss = {1:.4f}'.format(step, train_loss), end="\t")

  if step <= 1 or (step < 100 and step % 20 == 0) or step % eval_interval == 0:
    avg_return, cullsteps = compute_avg_return(eval_env, agent.policy, num_eval_episodes)
    print('step = {0}: average return = {1:.3f} cullsteps = {2:.1f}'.format(step, avg_return.numpy().item(), cullsteps))
    returns.append(avg_return)

Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.foldr(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.foldr(fn, elems))


Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.foldr(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.foldr(fn, elems))


step = 1: loss = 5893.6016	step = 1: average return = -687.800 cullsteps = 1.4
step = 20: average return = -582.200 cullsteps = 1.3
step = 40: average return = -703.400 cullsteps = 1.1
step = 60: average return = -683.400 cullsteps = 1.4
step = 80: average return = -734.800 cullsteps = 1.4
step = 100: loss = 0.0000	step = 200: loss = 960.8949	step = 300: loss = 0.0000	step = 400: loss = 39057.0039	step = 400: average return = -669.000 cullsteps = 1.0
step = 500: loss = 41985.3086	step = 600: loss = 0.0000	step = 700: loss = 81268.5234	step = 800: loss = 25637.8945	step = 800: average return = -752.600 cullsteps = 1.1
step = 900: loss = 0.0001	step = 1000: loss = 957.6707	step = 1100: loss = 907.1018	step = 1200: loss = 0.0001	step = 1200: average return = -618.600 cullsteps = 1.1
step = 1300: loss = 0.0001	step = 1400: loss = 956.4066	step = 1500: loss = 0.0002	step = 1600: loss = 30627.2422	step = 1600: average return = -1046.000 cullsteps = 8.2
step = 1700: loss = 25604.6484	step = 1

In [25]:
learned_reward, culleps = compute_avg_return(eval_env, agent.policy, num_episodes=100)
print ("reward of learned policy: ", learned_reward.numpy(), "culleps=", culleps)

reward of learned policy:  [-673.6] culleps= 1.25


In [26]:
init_ts = eval_env.reset()

def get_action(obs):
    """ execute the learned policy network 
       obs:  one float for each herd - the time since last culling
    """
    _ts = TimeStep(tf.constant([0.]),
                   tf.constant([0.]),
                   tf.constant([1]),
                   tf.constant([obs]))
    # a = agent.collect_policy.action(_ts) # just to see how much is explored versus exploited
    a = agent.policy.action(_ts)
    return a.action.numpy().item()

In [27]:
# what the learned policy does on a grid of observations (5 steps per row&col)
A = [[get_action([x,y])
 for y in np.arange(0.,1.,.05,np.float32)]
 for x in np.arange(0.,1.,.05,np.float32)]
A

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3],
 [0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3],
 [0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3],
 [0, 0, 0, 0, 0, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3],
 [0, 0, 0, 0, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3],
 [0, 0, 0, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3],
 [0, 0, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3],
 [0, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3],
 [0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3],
 [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3],
 [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3],
 [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3],
 [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3],
 [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3],
 [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,

### Play with parameters of manually designed q_network policy

In [28]:
W, b = agent.policy.trainable_variables
W = W.numpy()
b = b.numpy()
print ("weights\n", W, "\nbias", b)

weights
 [[-11.623947 -11.389742 -11.561311 -10.807395]
 [-11.50551  -11.517058 -11.269588 -10.697644]] 
bias [-13.105408  -13.213329  -13.2072115 -13.504318 ]


In [29]:
def nn(obs):
    y = np.dot(obs, W)+b
    return y

In [30]:
nn([0.1,.9])

array([-24.62276173, -24.71765604, -24.50597219, -24.21293755])